In [96]:
#spark home setup in CMD before running the below code. 

#setx SPARK_HOME C:\Users\VV232LY\spark3   

import findspark
findspark.init()

In [97]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField,IntegerType,FloatType,StringType
#from pyspark.sql.types import *
spark = SparkSession.builder.getOrCreate()

In [98]:
import json
retail_schemas = json.load(open('C:/Users/VV232LY/Learning/pyspark/data-master/data-master/retail_db_schema.json'))


In [4]:
columns = []
for i in retail_schemas['orders']:
    #print(i['column_name'])
    columns.append(i['column_name'])
    
print(columns)

['order_id', 'order_date', 'order_customer_id', 'order_status']


In [ ]:
#retail_schemas['orders']
columns = list(map(lambda col: col['column_name'], retail_schemas['orders']))
print(columns)

In [5]:
import pandas as pd
input = pd.read_csv('C:/Users/VV232LY/Learning/pyspark/data-master/data-master/retail_db/orders/part-00000', names=columns)

In [7]:
orders = spark.createDataFrame(input)


In [15]:
import re
df=orders.toDF(*[re.sub('[^A-Za-z0-9_]','',col) for col in orders.columns])

df.show()

+--------+--------------------+-----------------+---------------+
|order_id|          order_date|order_customer_id|   order_status|
+--------+--------------------+-----------------+---------------+
|       1|2013-07-25 00:00:...|            11599|         CLOSED|
|       2|2013-07-25 00:00:...|              256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:...|            12111|       COMPLETE|
|       4|2013-07-25 00:00:...|             8827|         CLOSED|
|       5|2013-07-25 00:00:...|            11318|       COMPLETE|
|       6|2013-07-25 00:00:...|             7130|       COMPLETE|
|       7|2013-07-25 00:00:...|             4530|       COMPLETE|
|       8|2013-07-25 00:00:...|             2911|     PROCESSING|
|       9|2013-07-25 00:00:...|             5657|PENDING_PAYMENT|
|      10|2013-07-25 00:00:...|             5648|PENDING_PAYMENT|
|      11|2013-07-25 00:00:...|              918| PAYMENT_REVIEW|
|      12|2013-07-25 00:00:...|             1837|         CLOSED|
|      13|

In [ ]:
#Get Daily Product Revenue 
1. One to Many relationship b/w orders and order_items

In [94]:
columns_order_items = ""
schema = ""
for i in retail_schemas['order_items']:
    schema = schema +" "+ i['column_name'] +" "+ i['data_type'] + ","
    
columns_order_items = schema[:-1]
print(columns_order_items)

 order_item_id int, order_item_order_id int, order_item_product_id int, order_item_quantity int, order_item_subtotal float, order_item_product_price float


In [91]:
order_item_schema = StructType(
                    [StructField("order_item_id", IntegerType(), True) ,
                     StructField("order_item_order_id", IntegerType(), True) ,
                     StructField("order_item_product_id", IntegerType(), True) ,
                     StructField("order_item_quantity", IntegerType(), True) ,
                     StructField("order_item_subtotal", FloatType(), True) ,
                     StructField("order_item_product_price", FloatType(), True) 
                    ]
                    )

#order_item_schema ='order_item_id int, order_item_order_id int,order_item_product_id int, order_item_quantity int, order_item_subtotal float,order_item_product_price float'

In [92]:
inputdata= spark.read \
            .format('csv') \
            .option('sep',',') \
            .option('header','false') \
            .option('Path','C:/Users/VV232LY/Learning/pyspark/data-master/data-master/retail_db/order_items/part-00000.csv') \
            .schema(order_item_schema) \
            .load()


In [107]:
orders_json= spark.read \
            .format('json') \
            .option('sep',',') \
            .option('header','false') \
            .option('Path','C:/Users/VV232LY/Learning/pyspark/data-master/data-master/retail_db_json/orders/part-r-00000.json') \
            .load()


In [108]:
orders_json.show()

+-----------------+--------------------+--------+---------------+
|order_customer_id|          order_date|order_id|   order_status|
+-----------------+--------------------+--------+---------------+
|            11599|2013-07-25 00:00:...|       1|         CLOSED|
|              256|2013-07-25 00:00:...|       2|PENDING_PAYMENT|
|            12111|2013-07-25 00:00:...|       3|       COMPLETE|
|             8827|2013-07-25 00:00:...|       4|         CLOSED|
|            11318|2013-07-25 00:00:...|       5|       COMPLETE|
|             7130|2013-07-25 00:00:...|       6|       COMPLETE|
|             4530|2013-07-25 00:00:...|       7|       COMPLETE|
|             2911|2013-07-25 00:00:...|       8|     PROCESSING|
|             5657|2013-07-25 00:00:...|       9|PENDING_PAYMENT|
|             5648|2013-07-25 00:00:...|      10|PENDING_PAYMENT|
|              918|2013-07-25 00:00:...|      11| PAYMENT_REVIEW|
|             1837|2013-07-25 00:00:...|      12|         CLOSED|
|         

In [ ]:
###Spark Read from mysql:

df=spark.read.format("jdbc") \
             .option("driver","com.mysql.cj.jdbc.Driver") \
             .option("user",username) \
             .option("password",password) \
             .option("url","jdbc:mysql://Mysql.dbmstutorials.com:3306") \
             .option("dbtable","tutorial_db.department") \
             .load()

##Read with Custom query: 

df=spark.read.format("jdbc").options(driver="com.mysql.cj.jdbc.Driver",
                                     user="tutorial_user",
                                     password="user_password",
                                     url="jdbc:mysql://Mysql.dbmstutorials.com:3306?serverTimezone=UTC&useSSL=false",
                                     query="SELECT * FROM tutorial_db.department WHERE dept_no IN (100,200)"
                                     ).load()

In [113]:
product_schema = StructType(
                    [StructField("product_id", IntegerType(), True) ,
                     StructField("product_category_id", IntegerType(), True) ,
                     StructField("product_name", StringType(), True) ,
                     StructField("product_description", StringType(), True) ,
                     StructField("product_price", FloatType(), True) ,
                     StructField("product_image", StringType(), True) 
                    ]
                    )


#order_item_schema ='order_item_id int, order_item_order_id int,order_item_product_id int, order_item_quantity int, order_item_subtotal float,order_item_product_price float'

In [115]:
columns_products = ""
schema = ""
for i in retail_schemas['products']:
    schema = schema +" "+ i['column_name'] +" "+ i['data_type'] + ","
    
columns_products = schema[:-1]
print(columns_products)

 product_id int, product_category_id int, product_name string, product_description string, product_price float, product_image string


In [116]:
product= spark.read \
            .format('csv') \
            .option('sep',',') \
            .option('header','false') \
            .option('Path','C:/Users/VV232LY/Learning/pyspark/data-master/data-master/retail_db/products/part-00000.csv') \
            .schema(columns_products) \
            .load()

product.show()


+----------+-------------------+--------------------+-------------------+-------------+--------------------+
|product_id|product_category_id|        product_name|product_description|product_price|       product_image|
+----------+-------------------+--------------------+-------------------+-------------+--------------------+
|         1|                  2|Quest Q64 10 FT. ...|               null|        59.98|http://images.acm...|
|         2|                  2|Under Armour Men'...|               null|       129.99|http://images.acm...|
|         3|                  2|Under Armour Men'...|               null|        89.99|http://images.acm...|
|         4|                  2|Under Armour Men'...|               null|        89.99|http://images.acm...|
|         5|                  2|Riddell Youth Rev...|               null|       199.99|http://images.acm...|
|         6|                  2|Jordan Men's VI R...|               null|       134.99|http://images.acm...|
|         7|       

In [117]:
product_json= spark.read \
            .format('json') \
            .option('sep',',') \
            .option('header','false') \
            .option('Path','C:/Users/VV232LY/Learning/pyspark/data-master/data-master/retail_db_json/products/part-r-00000') \
            .load()

product_json.show()


+-------------------+-------------------+----------+--------------------+--------------------+-------------+
|product_category_id|product_description|product_id|       product_image|        product_name|product_price|
+-------------------+-------------------+----------+--------------------+--------------------+-------------+
|                  2|                   |         1|http://images.acm...|Quest Q64 10 FT. ...|        59.98|
|                  2|                   |         2|http://images.acm...|Under Armour Men'...|       129.99|
|                  2|                   |         3|http://images.acm...|Under Armour Men'...|        89.99|
|                  2|                   |         4|http://images.acm...|Under Armour Men'...|        89.99|
|                  2|                   |         5|http://images.acm...|Riddell Youth Rev...|       199.99|
|                  2|                   |         6|http://images.acm...|Jordan Men's VI R...|       134.99|
|                  

In [120]:
product_json.createOrReplaceTempView("products")

In [125]:

query = """
select product_name,avg(product_price) from products
group by product_name
"""
print(type(query))
output = spark.sql(query)

output.show()

<class 'str'>
+--------------------+------------------+
|        product_name|avg(product_price)|
+--------------------+------------------+
|Quest Q64 10 FT. ...|59.980000000000004|
|Under Armour Men'...|129.98999999999995|
|Under Armour Men'...|             89.99|
|Riddell Youth Rev...|            199.99|
|Jordan Men's VI R...|            134.99|
|Schutt Youth Recr...|             99.99|
|Nike Men's Vapor ...|            129.99|
|Nike Adult Vapor ...|              50.0|
|Fitness Gear 300 ...|            209.99|
|Under Armour Men'...|            139.99|
|Quik Shade Summit...|            199.99|
|Under Armour Kids...|             59.99|
|Riddell Youth 360...|            299.99|
|Reebok Men's Full...|             29.97|
|Nike Men's Finger...|124.98999999999998|
|Under Armour Kids...|             54.99|
|Kijaro Dual Lock ...|29.990000000000002|
|Under Armour Men'...|            139.99|
|Elevation Trainin...|             79.99|
|Nike Men's USA Wh...|              90.0|
+-------------------